In [4]:
import numpy as np
import fitsio
import matplotlib.pyplot as plt
import os
import pandas as pd
from astropy.io import fits

In [5]:
def create_fits_TRAN(ID,files,classification):
    path = './data/'+classification+'/'
    if not os.path.isdir(path):
        os.mkdir(path)
    
    filename = './data/{}/{}.fits'.format(classification,ID)
    
    if os.path.isfile(filename):
#         print("Object {} already exists!".format(ID))
        return False
    
    df = pd.read_table('../tao_data/transient_info.txt')
    CRTS_ID = np.array(df['#CRTS ID'])
    if sum(CRTS_ID == ID) != 0:
        RA = np.array(df[CRTS_ID == ID]['RA (J2000)'])[0]
        DEC = np.array(df[CRTS_ID == ID]['Dec (J2000)'])[0]
        UT_Date = np.array(df[CRTS_ID == ID]['UT Date'])[0]
        MAG = np.array(df[CRTS_ID == ID]['Mag'])[0]
        CSS_Images = np.array(df[CRTS_ID == ID]['CSS images'])[0]
        SDSS = np.array(df[CRTS_ID == ID]['SDSS'])[0]
        OTHERS = np.array(df[CRTS_ID == ID]['Others'])[0]
        FOLLOWED = np.array(df[CRTS_ID == ID]['Followed'])[0]
        LAST = np.array(df[CRTS_ID == ID]['Last'])[0]
        LC = np.array(df[CRTS_ID == ID]['LC'])[0]
        FC = np.array(df[CRTS_ID == ID]['FC'])[0]
        CLASS = np.array(df[CRTS_ID == ID]['Classification'])[0]

        N = len(files)
        SEQUENCE = np.zeros(N, dtype='S2')
        DATE = np.zeros(N, dtype='S7')
        FIELD_ID = np.zeros(N, dtype='S6')
        OBS_IN_SEQ =  np.zeros(N, dtype='S4')
        CUTOUT = np.zeros(N, dtype='S4')
        HDU_EXT = np.arange(2,N+2).astype('S4')
        MJD = np.zeros(N, dtype='S11')
        for i, file in enumerate(files):
            try:
                hdul = fits.open('{}.fits'.format(file[:-12]))
                MJD[i] = str(hdul[0].header['MJD'])
            except:
                return False
            SEQUENCE[i] = file.split('/')[5].split('_')[0]
            DATE[i] = file.split('/')[5].split('_')[1]
            FIELD_ID[i] = file.split('/')[5].split('_')[2]
            OBS_IN_SEQ[i] = file.split('/')[5].split('_')[3]
            CUTOUT[i] = file.split('/')[5].split('_')[5]


        hdict = {'CRTS_ID': ID, 'RA (J2000)': RA, 'Dec (J2000)':DEC, 'N_images': N, 
                'UT Date': UT_Date, 'Mag': MAG, 'CSS images': CSS_Images,
                 'SDSS': SDSS, 'Others': OTHERS, 'Followed': FOLLOWED, 'Last': LAST,
                 'LC': LC, 'FC': FC, 'Classification': CLASS}
        data = {'HDU_Ext': HDU_EXT, 'Sequence': SEQUENCE, 'Date': DATE, 'MJD': MJD, 'Field_ID':FIELD_ID,
                'Obs_In_Seq': OBS_IN_SEQ, 'Cutout': CUTOUT}
        fitsio.write(filename, data, header=hdict)

        for file in files:
            img = np.load(file)
            fitsio.write(filename, img)
        return True
    else:
        print('{} Do not found in MANTRA'.format(ID))
        return False

In [6]:
classes = ['BZ']
for classification in classes:
    print(classification)
    path = '../tao_data/Test/{}/'.format(classification)
    objects = !ls $path
    print(len(objects))
    for objct in objects:
        path_file = path+objct+'/*.npy'
        files = !ls $path_file
        ID = objct[objct.find('CSS'):objct.find('CSS')+23]
        value = create_fits_TRAN(ID,files,classification)
    path = '../tao_data/Complete/{}/'.format(classification)
    objects = !ls $path
    print(len(objects))
    for objct in objects:
        path_file = path+objct+'/*.npy'
        files = !ls $path_file
        ID = objct[objct.find('CSS'):objct.find('CSS')+23]
        value = create_fits_TRAN(ID,files,classification)        

BZ
14


In [ ]:
def create_fits_NON(ID,files,path_file,classification):
    path = './data/NON/'
    if not os.path.isdir(path):
        os.mkdir(path)
    
    filename = './data/{}/{}.fits'.format(classification,ID)
    
    if os.path.isfile(filename):
#         print("Object {} already exists!".format(ID))
        return False
    
    df = pd.read_pickle('../mantra_data/NON_dataframe_info.pkl')
    CRTS_ID = np.array(df['Neg_ID'])
    if sum(CRTS_ID == ID) != 0:
        RA = np.array(df[ (CRTS_ID == ID)]['RA'])
        DEC = np.array(df[ (CRTS_ID == ID)]['DEC'])
        IMG_ref = np.array(df[ (CRTS_ID == ID)]['Im_ref'])[0]
        HDU_EXT = np.arange(2,N+2).astype('S4')

        hdict = {'CTRS_ID': ID, 'N_images': N, 'IMG_Ref': IMG_ref}
        data = {'HDU_Ext': HDU_EXT, 'RA (J2000)': RA, 'Dec (J2000)':DEC}
        fitsio.write(filename, data, header=hdict)

        for file in files:
            img = np.load(path_file+file)
            fitsio.write(filename, img)
        return True
    else:
        print('{} Do not found in MANTRA'.format(ID))
        return False

In [ ]:
classes =  ['NON']
for classification in classes:
    path = '../mantra_data/{}/'.format(classification)
    objects = !ls $path
    for objct in objects:
        path_file = path+objct+'/'
        files = !ls $path_file
        ID = objct[objct.find('CSS'):objct.find('CSS')+23]
        value = create_fits_NON(ID,files,path_file,classification)